In [33]:
import DBcm

In [34]:
db_details = "../src/db_utils/SwimDB.sqlite3"

In [35]:
with DBcm.UseDatabase(db_details) as db:
    db.execute("pragma table_list")
    resuts = db.fetchall()

In [36]:
resuts

[('main', 'times', 'table', 4, 0, 0),
 ('main', 'events', 'table', 3, 0, 0),
 ('main', 'sqlite_sequence', 'table', 2, 0, 0),
 ('main', 'swimmers', 'table', 3, 0, 0),
 ('main', 'sqlite_schema', 'table', 5, 0, 0),
 ('temp', 'sqlite_temp_schema', 'table', 5, 0, 0)]

In [37]:
SQL_SWIMMERS = """
    create table if not exists swimmers(
        id integer not null primary key autoincrement,
        name varchar(32) not null,
        age integer not null
    )
"""

SQL_EVENTS = """
    create table if not exists events(
        id integer not null primary key autoincrement,
        distance varchar(16) not null,
        stroke varchart(16) not null
    )
"""

SQL_TIMES = """
    create table if not exists times(
        swimmer_id integer not null,
        event_id integer not null,
        time varchart(16),
        ts timestamp default current_timestamp
    )
"""

with DBcm.UseDatabase(config=db_details) as db:
    db.execute(SQL_SWIMMERS)
    db.execute(SQL_EVENTS)
    db.execute(SQL_TIMES)
    db.execute("pragma table_list")
    result = db.fetchall()

In [38]:
result

[('main', 'times', 'table', 4, 0, 0),
 ('main', 'events', 'table', 3, 0, 0),
 ('main', 'sqlite_sequence', 'table', 2, 0, 0),
 ('main', 'swimmers', 'table', 3, 0, 0),
 ('main', 'sqlite_schema', 'table', 5, 0, 0),
 ('temp', 'sqlite_temp_schema', 'table', 5, 0, 0)]

## Populate tables

In [23]:
import os

FOLDER = "../src/swimdata/session1"

files = os.listdir(FOLDER)
files.remove(".DS_Store")

In [24]:
file_name = files[0]
name, age, distance, stroke = file_name.removesuffix(".txt").split("-")

In [25]:
SQL_INSERT_SWIMMER = """
    insert into swimmers
    (name, age)
    values
    (?,?)
"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_INSERT_SWIMMER, (name, age))
    db.execute("pragma table_list")
    result = db.fetchall()
    
result

[('main', 'times', 'table', 4, 0, 0),
 ('main', 'events', 'table', 3, 0, 0),
 ('main', 'sqlite_sequence', 'table', 2, 0, 0),
 ('main', 'swimmers', 'table', 3, 0, 0),
 ('main', 'sqlite_schema', 'table', 5, 0, 0),
 ('temp', 'sqlite_temp_schema', 'table', 5, 0, 0)]

In [26]:
SQL_SELECT_SWIMMER = """
    select *
    from swimmers
    where name = ? and age = ?;
"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_SELECT_SWIMMER, (name, age))
    result = db.fetchall()

result
if result:
    print("true")
else:
    print("false")

true


In [27]:
SQL_DELETE = "delete from times;"

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_DELETE)
    db.execute(SQL_SELECT_SWIMMER, (name, age))
    result = db.fetchall()

result

[(1, 'Abi', 10), (24, 'Abi', 10)]

In [28]:

with DBcm.UseDatabase(db_details) as db:
    for file_name in files:
        name, age, *_ = file_name.removesuffix(".txt").split("-")
        db.execute(SQL_SELECT_SWIMMER, (name, age))
        result = db.fetchall()
        if result:
            continue
        db.execute(SQL_INSERT_SWIMMER, (name, age))

In [29]:
SQL_INSERT_EVENT = """
    insert into events
    (distance, stroke) 
    values
    (?, ?);
"""

SQL_SELECT_EVENT = """
    select *
    from events
    where distance = ? and stroke = ?;
"""

In [30]:
with DBcm.UseDatabase(db_details) as db:
    for file_name in files:
        *_, distance, stroke = file_name.removesuffix(".txt").split("-")
        db.execute(SQL_SELECT_EVENT, (distance, stroke))
        result = db.fetchall()
        if result:
            continue
        db.execute(SQL_INSERT_EVENT, (distance, stroke))

In [31]:
SQL_GET_EVENT_ID = """
    select id
    from events
    where distance = ? and stroke = ?;
"""

SQL_GET_SWIMMER_ID = """
    select id
    from swimmers
    where name = ? and age = ?;
"""

SQL_INSERT_TIME = """
    insert into times
    (swimmer_id, event_id, time)
    values
    (?,?,?);
"""

In [39]:
with DBcm.UseDatabase(db_details) as db:
    for file_name in files:
        name, age, distance, stroke = file_name.removesuffix(".txt").split("-")

        db.execute(SQL_GET_SWIMMER_ID, (name, age))
        result = db.fetchone()
        swimmer_id = result[0]

        db.execute(SQL_GET_EVENT_ID, (distance, stroke))
        result = db.fetchone()
        event_id = result[0]

        with open(FOLDER + "/" + file_name, "r") as file:
            lines = file.readlines()
        times = lines[0].strip().split(",")

        for time in times:
            db.execute(SQL_INSERT_TIME, (swimmer_id, event_id, time))